In [1]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import List
import json
import time
from statistics import mean
import numpy as np

rootpath = ".."

import transfer_logic.transfer_LD_weightinit as transfer_LD_weightinit
import transfer_logic.transfer_LD_ft as transfer_LD_ft
import transfer_logic.transfer_autoLSTM as transfer_autoLSTM
import transfer_logic.transfer_GRU_weightinit as transfer_GRU_weightinit
import transfer_logic.transfer_autoGRU as transfer_autoGRU


degree_sign = u'\N{DEGREE SIGN}'

2023-10-29 02:45:03.838216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
result_filename = f"{rootpath}/results/result_data/transfer_results.json"

with open(result_filename, "r") as f:
    results_data = json.load(f)

ft_percents = results_data["options"]["ft_percents"]
shuffle_seeds = results_data["options"]["seeds"]
transfer_methods = ["weight_initialization_LSTMDense", "weight_initialization_Dense", "weight_initialization_AutoLSTM"]

In [3]:
def save_results(from_key, to_key, transfer_method, ft_percent, shuffle_seed, rmse=None, mae=None, mae_sd=None, training_time=None, datapoints=None):
    result_filename = f"{rootpath}/results/result_data/transfer_results.json"
    with open(result_filename, "r") as f:
        results_data = json.load(f)

    ft_percents = results_data["options"]["ft_percents"]
    shuffle_seeds = results_data["options"]["seeds"]

    if not to_key in results_data[transfer_method][from_key]:
        results_data[transfer_method][from_key][to_key] = {f"ft{int(x*100)}" : {
                    "rmse": [None]*len(shuffle_seeds),
                    "mae": [None]*len(shuffle_seeds),
                    "mae_sd": [None]*len(shuffle_seeds),
                    "training_time": [None]*len(shuffle_seeds),
                    "datapoints": 0
                } for x in ft_percents}
    
    saving_index = shuffle_seeds.index(shuffle_seed)
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["rmse"][saving_index] = rmse
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["mae"][saving_index] = mae
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["mae_sd"][saving_index] = mae_sd
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["training_time"][saving_index] = training_time
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["datapoints"] = datapoints
    
    with open(result_filename, "w") as f:
        json.dump(results_data, f, indent=4)

In [4]:
def plot_multiseed_error_pir(ft_percents: List[float], from_building: str, from_tower: int, from_season: str, to_building: str, to_tower: int, to_season: str = None, skip_existing_plot: bool = True):
    """
    Performs the following steps
    1. Performs transfer learning by finetuning entire model
    2. Performs transfer learning by finetuning only the output layer
    3. Plots mean absolute errors for test datasets picked with 5 different random seeds for both transfer learning methods
    4. Compares errors from transfer with error in the base model of the tower being transferred to
    5. Plots performance improvement ratio
    """
    to_features = [f'{to_building}_Tower_{to_tower} enteringWaterTemp', 
                    f'{to_building}_Tower_{to_tower} outdoorAirDryBulb', 
                    f'{to_building}_Tower_{to_tower} outdoorAirWetBulb', 
                    f'{to_building}_Tower_{to_tower} vfdPercent', 
                    f'{to_building}_Tower_{to_tower} pressureDiffEvap', 
                    f'{to_building}_Tower_{to_tower} pressureDiffCond', 
                    f'{to_building}_Tower_{to_tower} dayOfWeek', 
                    f'{to_building}_Tower_{to_tower} hourOfDay']
    to_target = f'{to_building}_Tower_{to_tower} energykWh'

    for ft_percent in ft_percents:
            for s, seed in enumerate(shuffle_seeds):
                # perform each transfer learning method

                # finetuning all layers
                rmse1, _, mae1, tt, dp = transfer_LD_weightinit.transfer_LD_weightinit(from_building_name=from_building, from_tower_number=from_tower, from_season=from_season, to_building_name=to_building, to_tower_number=to_tower, to_features=to_features, to_target=to_target, to_season=to_season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
                save_results(from_key=f"{from_building}{from_tower}_{from_season}", to_key=f"{to_building}{to_tower}_{to_season}", transfer_method=transfer_methods[0], ft_percent=ft_percent, shuffle_seed=seed, rmse=rmse1, mae=mae1, training_time=tt, datapoints=dp)

                # finetuning only output layer
                rmse2, _, mae2, tt, dp = transfer_LD_ft.transfer_LD_dense(from_building_name=from_building, from_tower_number=from_tower, from_season=from_season, to_building_name=to_building, to_tower_number=to_tower, to_features=to_features, to_target=to_target, to_season=to_season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
                save_results(from_key=f"{from_building}{from_tower}_{from_season}", to_key=f"{to_building}{to_tower}_{to_season}", transfer_method=transfer_methods[1], ft_percent=ft_percent, shuffle_seed=seed, rmse=rmse2, mae=mae2, training_time=tt, datapoints=dp)

                # using lstm autoencoder model and finetuning only lstm layer
                rmse3, _, mae3, tt, dp = transfer_autoLSTM.transfer_autoLSTM(from_building_name=from_building, from_tower_number=from_tower, from_season=from_season, to_building_name=to_building, to_tower_number=to_tower, to_features=to_features, to_target=to_target, to_season=to_season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
                save_results(from_key=f"{from_building}{from_tower}_{from_season}", to_key=f"{to_building}{to_tower}_{to_season}", transfer_method=transfer_methods[2], ft_percent=ft_percent, shuffle_seed=seed, rmse=rmse3, mae=mae3, training_time=tt, datapoints=dp)

# Intra-season (Summer)

In [5]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="ESB", from_tower=1, from_season="summer", to_building="ESB", to_tower=2, to_season="summer")

Tower 2 first temp: 8.929999351501465


2023-10-29 02:45:07.513668: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


352/352 [==============================] - 1s 917us/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 1s 901us/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 0s 998us/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 1s 896us/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 1s 965us/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 0s 987us/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 1s 963us/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 1s 1ms/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 1s 1ms/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 1s 902us/step
Tower 2 first temp: 8.929999351501465
352/352 [==============================] - 1s 999us/st

INFO:tensorflow:Assets written to: ../results/models_saved/lstmdense_ft/esb1summer_to_esb2summer_ft20_seed17/assets


INFO:tensorflow:Assets written to: ../results/models_saved/lstmdense_ft/esb1summer_to_esb2summer_ft20_seed17/assets


...

INFO:tensorflow:Assets written to: ../results/models_saved/dense_ft/esb1summer_to_esb2summer_ft80_seed98/assets


INFO:tensorflow:Assets written to: ../results/models_saved/dense_ft/esb1summer_to_esb2summer_ft80_seed98/assets


71/71 [==============================] - 0s 993us/step
Tower 2 first temp: 8.929999351501465
Finetuning for ft=0.8 seed=98
INFO:tensorflow:Assets written to: ../results/models_saved/eld_ft/esb1summer_to_esb2summer_ft80_seed98/assets


INFO:tensorflow:Assets written to: ../results/models_saved/eld_ft/esb1summer_to_esb2summer_ft80_seed98/assets


71/71 [==============================] - 0s 1ms/step


In [ ]:
# create all combinations of intraseason transfers for Kissam and ESB
buildings = ["ESB", "Kissam"]
towers = [1, 2]
seasons = ["summer", "fall"]
intraseason_combinations = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings for s2 in seasons for t2 in towers for b2 in buildings if (b1,t1,s1)!=(b2,t2,s2) and s1==s2]
intraseason_combinations = list(set(intraseason_combinations)) # making sure there are no duplicates

# TEMPORARY ADDITION FIXME
import random
random.seed(17)
intraseason_combinations = random.sample(intraseason_combinations, 5)
print(intraseason_combinations)

In [ ]:
for c in [intraseason_combinations[0]]:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)

# Inter-season (Summer to Fall, Fall to Summer)

### Kissam and ESB

In [ ]:
# create all combinations of interseason transfers between summer and fall for Kissam and ESB
buildings = ["ESB", "Kissam"]
towers = [1, 2]
seasons = ["summer", "fall"]
interseason_combinations = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings for s2 in seasons for t2 in towers for b2 in buildings if (b1,t1,s1)!=(b2,t2,s2) and s1!=s2]
interseason_combinations = list(set(interseason_combinations)) # making sure there are no duplicates

# TEMPORARY ADDITION FIXME
interseason_combinations = random.sample(interseason_combinations, 5)
print(interseason_combinations)

In [ ]:
for c in interseason_combinations:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)

## Display graphs

In [ ]:
# Create a subplots figure with two subplots and set title
x = [fp * 100 for fp in ft_percents]

for tm in results_data:
    if tm == "options": continue
    for from_key in results_data[tm]:
        for to_key in results_data[tm][from_key]:
            if to_key == "base": continue
            try:
                average_mae = [0.0] * len(ft_percents)
                for i, ft in enumerate([f"ft{int(fp * 100)}" for fp in ft_percents]):
                    average_mae[i] = results_data[tm][from_key][to_key][ft]["mae"]
                    if None in average_mae[i]: raise ValueError("Skip this graph")
                    
                
                average_mae = [[average_mae[j][i] for j in range(len(average_mae))] for i in range(len(average_mae[0]))] # reshape
                
                # add errors of transfer
                figfinal = go.Figure()
                for i, ft_mae_list in enumerate(average_mae):
                    figfinal.add_trace(
                        go.Scatter(
                            x=x, 
                            y=ft_mae_list, 
                            name=f"MAE for {shuffle_seeds[i]}")
                    )
                # add error of base model
                base_error_info = results_data[tm][to_key]["base"]
                figfinal.add_trace(go.Scatter(
                    x=x, 
                    y=[base_error_info["mae"]]*len(ft_percents), 
                    name=f"Base model MAE seed-42", 
                    line=dict(color='rgb(155, 185, 155)'),
                    mode='lines'))
                # add sd of base model
                y_upper = [base_error_info["mae"] + base_error_info["mae_sd"]]*len(ft_percents)
                y_lower = [base_error_info["mae"] - base_error_info["mae_sd"]]*len(ft_percents)
                figfinal.add_trace(go.Scatter(
                    x=x+x[::-1], # x, then x reversed
                    y=y_upper+y_lower[::-1], # upper, then lower reversed
                    fill='toself',
                    fillcolor='rgba(155, 185, 155,0.1)',
                    line=dict(color='rgba(0,0,0,0)'),
                    hoverinfo="skip",
                    showlegend=False
                ))
                
                # formatting
                figfinal.update_yaxes(range=[-1, 12])
                figfinal.update_layout(
                    title=f"{from_key} to {to_key} ({tm})",
                    xaxis_title='Amount of data',
                    yaxis_title='Mean absolute error'
                )
                # figfinal.show()
                figfinal.write_html(f"{rootpath}/results/plots/transfer_comparison/{tm}_{from_key}_to_{to_key}_mae.html")
            except: break # in case of null values (transfers that have not been run yet)